# 第 4 讲：求积问题

## Newton-Cotes

In [1]:
import numpy as np
import sympy as sy

In [2]:
def factorial(num):
    res = 1
    for x in range(1, num+1):
        res *= x
    return res

In [3]:
def mul(n, i, t):
    res = 1
    for j in range(0, n+1):
        if j != i:
            res *= (t - j)
    return res

In [4]:
def dt(a, b, i, f):
    t = np.linspace(a, b, 1000)
    return sum(((b-a) / 1000) * f(b-a, i, t))

In [5]:
def C(n, i):
    return ( (-1) ** (n - i) ) / (factorial(i) * factorial(n - i)) * dt(0, n, i, mul)

In [6]:
def f(x):
    return 1/ (1 + x)

In [7]:
def NC(f, xs, n):
    x = np.linspace(xvalues[0], xs[-1], n+1)
    h = x[1] - x[0]
    total = 0
    for i in range(0, n+1):
        total += C(n, i) * f(x[i])
    return h * total

In [8]:
xvalues = [0, 1]

In [9]:
NC(f, xvalues, 4)

0.6932314905381517

## 梯形公式

令Newton-Cotes的n变为1，我们就得到了梯形公式

递归写法

In [10]:
def trapezoidal(f, a, b, n):
    if n == 1:
        return (b-a) * (f(a) + f(b)) / 2
    else:
        if n % 2 == 0:
            h = (b-a) / n
            x = [a + i*h for i in range(n+1)]
        else:
            h = (b-a) / (n+1)
            x = [a + i*h for i in range(n+1)]
        s1 = trapezoidal(f, a, x[n//2], n//2)
        s2 = trapezoidal(f, x[n//2], b, n//2)
        return s1 + s2

In [11]:
trapezoidal(f, 0, 1, 4)

0.6970238095238095

## Romberg

In [12]:
def Romberg(f, a, b, n):
    R = [[0 for i in range(n + 1)] for j in range(n+1)]
    R[1][1] = 1/2 * (b - a) * ( f(a) + f(b) )
    # print(R[1][1])
    for i in range(2, n + 1):
        h = (b - a) / (2 ** (i - 1))
        R[i][1] = 1/2 * R[i-1][1] +  h * sum(( f(a + (2 * j - 1) * h ) for j in range(1, 2 ** (i - 2) + 1) ))
        print(f"{i} {1}")
        print(R[i][1])
        for k in range(2, i + 1):
            R[i][k] = (4 ** (k - 1) * R[i][k-1] - R[i-1][k-1]) / (4 ** (k - 1) - 1)
            print(f"{i} {k}")            
            print(R[i][k])

In [13]:
Romberg(f, 0, 3.14, 6)

2 1
1.5855084682042895
2 2
1.4642689398923538
3 1
1.4665092800847703
3 2
1.4268428840449305
3 3
1.4243478136551024
4 1
1.4326003111727352
4 2
1.4212973215353901
4 3
1.4209276173680874
4 4
1.4208733285381347
5 1
1.4237059353715606
5 2
1.4207411434378356
5 3
1.4207040648979985
5 4
1.4207005164460922
5 5
1.4206998387516137
6 1
1.4214505835272604
6 2
1.4206987995791602
6 3
1.4206959766552485
6 4
1.4206958482704428
6 5
1.4206958299638714
6 6
1.4206958260452127


## Gauss-Legendre

In [14]:
def Legendre(N, x):
    p0 = 1
    p1 = x
    sy.simplify(x)
    for i in range(1,N):
        p = (2*i+1)/(i+1)*x*p1 - i/(i+1)*p0
        p0 = p1
        p1 = p
    return sy.simplify(p1)

In [15]:
Legendre(2, sy.symbols('x'))

1.5*x**2 - 0.5

In [ ]:
print(sy.solve(Legendre(2, sy.symbols('x'))))
print(sy.solve(Legendre(3, sy.symbols('x'))))
print(sy.solve(Legendre(4, sy.symbols('x'))))
print(sy.solve(Legendre(5, sy.symbols('x'))))
print(sy.solve(Legendre(6, sy.symbols('x'))))
print(sy.solve(Legendre(7, sy.symbols('x'))))
print(sy.solve(Legendre(8, sy.symbols('x'))))

[-0.577350269189626, 0.577350269189626]
[-0.774596669241483, 0.0, 0.774596669241483]
[-0.861136311594053, -0.339981043584856, 0.339981043584856, 0.861136311594053]
[-0.906179845938664, -0.538469310105683, 0.0, 0.538469310105683, 0.906179845938664]
